**pip install arch:**
pip es una herramienta utilizada para instalar paquetes de Python desde el repositorio PyPI (Python Package Index).
**install** es el subcomando de pip que se utiliza para instalar paquetes.
**arch** es el nombre del paquete que se desea instalar.

In [2]:
pip install arch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.7/981.7 kB 7.7 MB/s eta 0:00:00


In [3]:

import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import seaborn as sns
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

## Loading the data

**yfinance.download():**

yfinance es una biblioteca de Python que permite descargar datos financieros de Yahoo Finance. En este caso, estás utilizando la función download() de esta biblioteca para descargar los datos.

**tickers="^GSPC ^FTSE ^N225 ^GDAXI":**

Este argumento especifica los símbolos de los índices bursátiles que deseas descargar. Estás descargando datos para los índices S&P 500, FTSE 100, Nikkei 225 y DAX.

**start="1994-01-07":**

Este argumento establece la fecha de inicio para la descarga de datos. En este caso, se inicia el 7 de enero de 1994.

**end="2018-01-29":**

Este argumento establece la fecha de finalización para la descarga de datos. En este caso, se finaliza el 29 de enero de 2018.

**interval="1d":**

Este argumento especifica el intervalo de tiempo de los datos que se descargarán. En este caso, se están descargando datos diarios.

**group_by='ticker':**

Este argumento agrupa los datos por el símbolo del índice bursátil al que pertenecen. Cada índice tendrá sus propias filas de datos.

**auto_adjust=True:**

Este argumento indica que los datos deben ser ajustados automáticamente para tener en cuenta eventos corporativos como divisiones de acciones y dividendos.

**threads=True:**

Este argumento habilita la descarga paralela de datos utilizando múltiples hilos. Esto puede acelerar la descarga de datos cuando se descargan datos de varios activos financieros al mismo tiempo.

In [5]:
raw_data = yfinance.download(tickers="^GSPC ^FTSE ^N225 ^GDAXI", start="1994-01-07", end="2018-01-29",
                             interval="1d", group_by='ticker', auto_adjust=True, threads=True)

[*********************100%%**********************]  4 of 4 completed


**raw_data** es el DataFrame original que contiene los datos financieros que descargaste previamente utilizando la biblioteca yfinance.

**df_comp** es una nueva variable que se crea para almacenar la copia de los datos. Es una copia independiente del DataFrame original, por lo que cualquier modificación que hagas en df_comp no afectará a raw_data y viceversa.

In [6]:
df_comp = raw_data.copy()

**df_comp['spx'] = df_comp['^GSPC'].Close[:]:**

Agrega una nueva columna llamada "spx" a df_comp.
Asigna los valores de cierre de la columna "Close" del índice S&P 500 (^GSPC) a la nueva columna "spx".

**df_comp['dax'] = df_comp['^GDAXI'].Close[:]:**

Agrega una nueva columna llamada "dax" a df_comp.
Asigna los valores de cierre de la columna "Close" del índice DAX (^GDAXI) a la nueva columna "dax".

**df_comp['ftse'] = df_comp['^FTSE'].Close[:]:**

Agrega una nueva columna llamada "ftse" a df_comp.
Asigna los valores de cierre de la columna "Close" del índice FTSE 100 (^FTSE) a la nueva columna "ftse".

**df_comp['nikkei'] = df_comp['^N225'].Close[:]:**

Agrega una nueva columna llamada "nikkei" a df_comp.
Asigna los valores de cierre de la columna "Close" del índice Nikkei 225 (^N225) a la nueva columna "nikkei".



In [7]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

**df_comp = df_comp.iloc[1:]:**

Esta línea elimina la primera fila del DataFrame df_comp, posiblemente eliminando la primera observación de datos. Esto puede ser útil si deseas eliminar filas con valores faltantes o simplemente omitir el primer período de tiempo en tus datos.

**del df_comp['^N225'], del df_comp['^GSPC'], del df_comp['^GDAXI'], del df_comp['^FTSE']:**

Estas líneas eliminan las columnas con los nombres indicados del DataFrame df_comp. En este caso, se están eliminando las columnas correspondientes a los índices bursátiles Nikkei 225, S&P 500, DAX y FTSE 100. Esto puede ser útil si no necesitas estas columnas en tu análisis posterior.

**df_comp = df_comp.asfreq('b'):**

Esta línea establece la frecuencia de muestreo de los datos en el DataFrame df_comp como "b", que generalmente se utiliza para indicar días hábiles (business days). Esto puede ser importante si deseas asegurarte de que tus datos estén muestreados en días hábiles y no incluyan los fines de semana.

**df_comp = df_comp.fillna(method='ffill'):**

Esta línea rellena los valores faltantes en el DataFrame df_comp utilizando el método 'ffill' (forward fill), lo que significa que los valores faltantes se llenan con el valor anterior válido en la misma columna. Esto es común en el análisis de series temporales para tratar los valores faltantes y asegurarse de que los datos estén completos y sin huecos.

In [8]:
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

## Creating Returns

 **df_comp['ret_spx'] = df_comp.spx.pct_change(1) * 100:**

Agrega una nueva columna llamada "ret_spx" a df_comp.
Calcula los retornos diarios del índice S&P 500 (columna "spx") utilizando la función pct_change(1). Esto calcula el cambio porcentual en comparación con el día anterior.
Multiplica por 100 para expresar los retornos en porcentaje.

**df_comp['ret_ftse'] = df_comp.ftse.pct_change(1) * 100:**

Agrega una nueva columna llamada "ret_ftse" a df_comp.
Calcula los retornos diarios del índice FTSE 100 (columna "ftse") de manera similar a la línea anterior.

**df_comp['ret_dax'] = df_comp.dax.pct_change(1) * 100:**

Agrega una nueva columna llamada "ret_dax" a df_comp.
Calcula los retornos diarios del índice DAX (columna "dax") de manera similar a las líneas anteriores.

**df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1) * 100:**

Agrega una nueva columna llamada "ret_nikkei" a df_comp.
Calcula los retornos diarios del índice Nikkei 225 (columna "nikkei") de manera similar a las líneas anteriores.

In [9]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

## Splitting the Data

**size = int(len(df_comp) * 0.8)**

Esta línea calcula el tamaño deseado para el conjunto de datos de entrenamiento (df) como el 80% del tamaño total del DataFrame df_comp. El 80% se calcula multiplicando la longitud total de df_comp por 0.8 y convirtiéndolo en un entero.

**df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]:**

En esta línea, se realiza la división del DataFrame df_comp en dos DataFrames diferentes: df y df_test.
df contiene las primeras filas del DataFrame df_comp, desde el principio hasta el índice size - 1. Estas serán las filas utilizadas para entrenar modelos y realizar análisis.
df_test contiene las filas desde el índice size hasta el final del DataFrame df_comp. Estas filas se reservan para realizar pruebas y evaluaciones posteriores.

In [10]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

## Fitting a Model

El comando pip install pmdarima se utiliza para instalar la biblioteca pmdarima, que proporciona herramientas para ajustar automáticamente modelos ARIMA (Autoregressive Integrated Moving Average) y SARIMA (Seasonal ARIMA) a series temporales.

In [12]:
pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.1 MB/s eta 0:00:00


La función auto_arima es útil para encontrar el mejor modelo ARIMA o SARIMA para tus datos sin necesidad de realizar una búsqueda exhaustiva manual de hiperparámetros.

In [13]:
from pmdarima.arima import auto_arima

**auto_arima** es una función que busca automáticamente los mejores hiperparámetros para un modelo ARIMA en base a la serie temporal proporcionada. Esta función intentará encontrar los valores óptimos de los órdenes (p, d, q) y (P, D, Q, s) para el modelo ARIMA o SARIMA, donde (p, d, q) son los órdenes del proceso autoregresivo, diferenciación y media móvil, respectivamente, y (P, D, Q, s) son los órdenes de la componente estacional, si corresponde.

**df.ret_ftse[1:]** selecciona la serie de retornos diarios del índice FTSE 100 desde la segunda observación en adelante. La razón para excluir la primera observación podría deberse a que los retornos del primer día no pueden calcularse debido a la falta de datos previos para comparar.

**model_auto** es una variable en la que se almacena el modelo ARIMA que ha sido ajustado automáticamente utilizando auto_arima.

In [15]:
model_auto = auto_arima(df.ret_ftse[1:])

La variable model_auto contiene el modelo ARIMA que fue ajustado automáticamente utilizando la función auto_arima.

In [16]:
model_auto

ARIMA(order=(4, 0, 5), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

La función **model_auto.summary()** se utiliza para obtener un resumen detallado de un modelo ARIMA

In [17]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5019
Model:               SARIMAX(4, 0, 5)   Log Likelihood               -7882.776
Date:                Fri, 13 Oct 2023   AIC                          15785.552
Time:                        18:39:07   BIC                          15850.762
Sample:                    01-11-1994   HQIC                         15808.403
                         - 04-05-2013                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0120      0.082      0.147      0.883      -0.148       0.172
ar.L2         -0.6543      0.077     -8.457      0.000      -0.806      -0.503
ar.L3         -0.1628      0.071     -2.290      0.022      -0.302      -0.023
ar.L4          0.2014      0.074      2.711      0.007       0.056       0.347
ma.L1         -0.0357      0.081     -0.440      0.660      -0.195       0.123
ma.L2          0.6068      0.078      7.769      0.000       0.454       0.760
ma.L3          0.0621      0.068      0.908      0.364      -0.072       0.196
ma.L4         -0.1933      0.073     -2.649      0.008      -0.336      -0.050
ma.L5         -0.1053      0.010    -11.067      0.000      -0.124      -0.087
sigma2         1.3543      0.015     91.437      0.000       1.325       1.383
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              6354.62
Prob(Q):                              0.96   Prob(JB):                         0.00
Heteroskedasticity (H):               1.99   Skew:                            -0.20
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.50
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## Important Arguments

**df_comp.ret_ftse[1:]:** Esta es la serie de retornos diarios del índice FTSE 100, excluyendo la primera observación. Se utiliza como serie temporal principal para el modelo SARIMA.

**exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:]:** Estas son las variables exógenas, es decir, las variables predictoras que se utilizan en el modelo. En este caso, se están utilizando los retornos diarios de los índices S&P 500, DAX y Nikkei 225 como variables exógenas para el modelo SARIMA.

**m = 5:** Este argumento especifica la frecuencia estacional de los datos. En este caso, se asume una estacionalidad de 5, lo que indica una estacionalidad semanal.

**max_order = None:** Este argumento establece el orden máximo del modelo SARIMA. Al establecerlo en None, la función buscará automáticamente el orden máximo en función de los datos proporcionados.

**max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2:** Estos argumentos establecen los máximos órdenes de los componentes autoregresivos, de media móvil y de diferenciación para el modelo SARIMA. Son valores máximos posibles para la búsqueda automática de hiperparámetros.

**maxiter = 50:** Este argumento establece el número máximo de iteraciones permitidas durante el proceso de búsqueda de hiperparámetros.

**alpha = 0.05:** Este es el nivel de significancia utilizado para la prueba de Box-Jenkins.

**n_jobs = -1:** Este argumento indica que se deben utilizar todos los núcleos disponibles en la búsqueda de hiperparámetros.

**trend = 'ct':** Esto especifica que se debe incluir una tendencia constante y lineal en el modelo SARIMA.

**information_criterion = 'oob':** Este argumento especifica el criterio de información utilizado para seleccionar el mejor modelo.

**out_of_sample = int(len(df_comp) * 0.2):** Esto indica el número de observaciones fuera de la muestra que se deben utilizar para evaluar el modelo.

In [18]:
model_auto = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob',
                       out_of_sample = int(len(df_comp)*0.2))


# !!! Important Note: In pdmarima v1.5.2, out_of_sample_size is replaced with out_of_sample, so make sure to use the latter!


# exogenous -> outside factors (e.g other time series)
# m -> seasonal cycle length
# max_order -> maximum amount of variables to be used in the regression (p + q)
# max_p -> maximum AR components
# max_q -> maximum MA components
# max_d -> maximum Integrations
# maxiter -> maximum iterations we're giving the model to converge the coefficients (becomes harder as the order increases)
# alpha -> level of significance, default is 5%, which we should be using most of the time
# n_jobs -> how many models to fit at a time (-1 indicates "as many as possible")
# trend -> "ct" usually
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob'
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple -> validates the model selection (pass the entire dataset, and set 20% to be the out_of_sampl

La función **model_auto.summary()** se utiliza para obtener un resumen detallado de un modelo ARIMA

In [19]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                 6274
Model:             SARIMAX(0, 0, 4)x(0, 0, [1], 5)   Log Likelihood               -9560.337
Date:                             Fri, 13 Oct 2023   AIC                          19136.673
Time:                                     19:05:46   BIC                          19190.627
Sample:                                 01-11-1994   HQIC                         19155.368
                                      - 01-26-2018                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0246      0.028      0.884      0.377      -0.030       0.079
drift      -2.025e-06   7.91e-06     -0.256      0.798   -1.75e-05    1.35e-05
ma.L1         -0.0201      0.008     -2.496      0.013      -0.036      -0.004
ma.L2         -0.0496      0.007     -6.925      0.000      -0.064      -0.036
ma.L3         -0.0694      0.007     -9.724      0.000      -0.083      -0.055
ma.L4          0.0333      0.008      4.385      0.000       0.018       0.048
ma.S.L5       -0.0513      0.007     -7.261      0.000      -0.065      -0.037
sigma2         1.2386      0.012    104.270      0.000       1.215       1.262
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              8783.36
Prob(Q):                              0.98   Prob(JB):                         0.00
Heteroskedasticity (H):               0.86   Skew:                            -0.17
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.79
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""